<a href="https://colab.research.google.com/github/LM1997610/Networking-for-Big-Data/blob/main/NBD_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBD - Challenge 2

In [237]:
import time; import datetime

import numpy as np
import pandas as pd
import random as rdm
from tqdm.auto import tqdm

#from tabulate import tabulate
from matplotlib import pyplot as plt

import math

In [238]:
!gdown 19baCZ9_wVNSBWVRJFtDIA4haBxv6IXAw

Downloading...
From: https://drive.google.com/uc?id=19baCZ9_wVNSBWVRJFtDIA4haBxv6IXAw
To: /content/Cell_a.zip
100% 43.3M/43.3M [00:00<00:00, 207MB/s]


### Data loading

In [239]:
data = pd.read_csv('Cell_a.zip')

In [240]:
print("\n data.shape : {}\n".format((data.shape))) #2329133
data.head()


 data.shape : (2329133, 5)



,Job_ID,Task_ID,Arrival_Time,CPU,Memory
0,375000667413,0,603026300,0.041851,0.001169
1,375000669289,0,606413041,0.024968,0.001179
2,375000670586,0,608994453,0.024176,0.001173
3,375000670588,0,608994466,0.019552,0.001163
4,375000670590,0,609042903,0.028044,0.001179


### Constants

In [241]:
N = 64
mu = 0.1

# Explorative data analysis

In [242]:
def eCCDF(job:int, field:str, d=data) -> np.array:
    sub = d[d['Job_ID']==job]
    arr = np.array(sub[field])
    quantiles, counts = np.unique(arr, return_counts=True)
    cumprob = np.cumsum(counts).astype(np.double)/arr.size
    return quantiles, 1-cumprob

In [ ]:
count = data.groupby('Job_ID')["Task_ID"].count()
mean_cpu = data.groupby('Job_ID')["CPU"].mean()
mean_memory = data.groupby('Job_ID')["Memory"].mean()
var_cpu = data.groupby('Job_ID')["CPU"].var()
var_memory = data.groupby('Job_ID')["Memory"].var()

In [ ]:
jobs = pd.unique(data['Job_ID'])
temp = {job:k for (job,k) in zip(jobs, count)}

multitask = [job for job in jobs if temp[job]>1]

eCCDF_CPU = {job:0 for job in jobs}
for job in tqdm(multitask):
    eCCDF_CPU[job] = list(zip(eCCDF(job,'CPU')))

eCCDF_memory = {job:0 for job in jobs}
for job in tqdm(multitask):
    eCCDF_memory[job] = list(zip(eCCDF(job,'Memory')))

100%|██████████| 70702/70702 [06:12<00:00, 189.93it/s]


In [ ]:
stats = pd.DataFrame({
    'Jobs':jobs,
    'Tasks_count':count,
    'Mean_CPU':mean_cpu,
    'Mean_Memory':mean_memory,
    'CPU_variance':var_cpu,
    'Memory_variance':var_memory,
    'CPU_eCCDF': [list(value) if value != 0 else value for value in eCCDF_CPU.values()],
    'Memory_eCCDF': [list(value) if value != 0 else value for value in eCCDF_memory.values()]
})

In [ ]:
print("\n stats.shape : {}\n".format((stats.shape)))

stats.head()


 stats.shape : (1163061, 8)



,Jobs,Tasks count,Mean CPU,Mean Memory,CPU variance,Memory variance,CPU eCCDF,Memory eCCDF
Job_ID,,,,,,,,
375000667413,375000667413,1,0.041851,0.001169,NaN,NaN,0,0
375000669289,375000669289,1,0.024968,0.001179,NaN,NaN,0,0
375000670586,375000670586,1,0.024176,0.001173,NaN,NaN,0,0
375000670588,375000670588,1,0.019552,0.001163,NaN,NaN,0,0
375000670590,375000670590,1,0.028044,0.001179,NaN,NaN,0,0


# Implementing the baselines: LWL for dispatching, FCFS for scheduling

## LWL policy for dispatching

## FCFS for scheduling

In [ ]:
!git clone https://github.com/LM1997610/Networking-for-Big-Data NBD

Cloning into 'NBD'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 47 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), 24.20 KiB | 1.61 MiB/s, done.


In [ ]:
queue_status = []

In [275]:
class real_FCFS:

    def __init__(self, server_id = int):   # constructor function  

        self.server_id = server_id   # non so se servirà

        self.memory =  1 #GNMU

        self.queue =  []  # will store just arrival_time and service time of task

        self.queue_status = []  # keep track of our measures

        self.workload = 0   # keep trak of work left behind / lateness

        self.list_done_task = []   # for messages with dispatcher 
    
    def is_empty(self):

      if not self.queue: return True
      else: return False      ### and do some work 
    
    def add_to_queue(self, dizionario : dict, µ = 0.1 ):

      new_job_id = str(int(dizionario["Job_ID"]))+ "_" + str(int(dizionario["Task_ID"]))

      arrival_time_task = int(dizionario["Arrival_Time"])

      running_time_task = (dizionario["CPU"]/ µ)*1000     # from seconds to milliseconds

      a_dict = {"id": new_job_id, "arrival_time": arrival_time_task, "service_time" : running_time_task}

      if self.memory > dizionario["Memory"]: 
         self.memory -= dizionario["Memory"]
         self.queue.append(a_dict)   ### and do some work 
         
      else: print("...not enough space in memory")  ## and think what to do 

## 
################################################################################
# time_delta fondamentali per capire se il server è impegnato oppure no

###############################################################################

    def exit_time(self): # cumulative execution times
      times = []

      for n, task in enumerate(self.queue):
        
        if n == 0: times.append(task['arrival_time'])

        else: times.append(times[n-1] + task['arrival_time'])

      self.queue_status = {x["id"]: [x["arrival_time"], x["service_time"], v] for x,v in zip(self.queue, times)}
      # KEYS = new_job_id --> VALUES = Arrival_Time, Burst_time, exit_time

      return self.queue_status
      
#########################################################################################

    def turn_around_times(self): 

        a_list = []

        for task in x.queue_status.keys():

            desired = x.queue_status[task][-1]-x.queue_status[task][0]

            # difference between Exit time and Arrival of the processes

            #if desired < 0: 
            #    desired = dizionario[i][1]
            # if negative there is no queue ahead, so TAT = Burst_time
            # perche WT = TAT - Burst_time deve essere  0
            # not sure 

            a_list.append(desired)

        self.queue_status = {v[0]:(*v[1], x) for v, x in zip(self.queue_status.items(), a_list)}
        # KEYS = new_job_id --> VALUES = Arrival_Time, Burst_time, Exit_time, Turn_around

        return  self.queue_status

#########################################################################################

    def waiting_times(self):
      
        a_list = []
        
        for task in x.queue_status.keys():
            
            wt = x.queue_status[task][-1]-x.queue_status[task][1]
            # difference between turn around time and burst time

            a_list.append(wt)
        
        self.queue_status = {v[0]:(*v[1], x) for v, x in zip(self.queue_status.items(), a_list)}
        # KEYS = new_job_id --> VALUES = Arrival_Time, Burst_time, Exit_time, Turn_around, Waiting

        return self.queue_status

    def main_scheduling(self):

      if self.is_empty() == False: 

        print("ho da lavorare")
        
        my_list = [x["arrival_time"] for x in x.queue]
        time_deltas = [my_list[i]-my_list[i-1] if i !=0 else 0 for i in reversed(range(len(my_list)))][::-1]
        time_deltas



In [284]:
x.main_scheduling()

ho da lavorare


In [281]:
x = real_FCFS(13)
print(x)

In [282]:
x.add_to_queue(data.iloc[0])
x.add_to_queue(data.iloc[1])
x.add_to_queue(data.iloc[2])
x.add_to_queue(data.iloc[3])

In [283]:
x.is_empty()

False

In [247]:
x.memory

0.9953155517578127

In [222]:
x.queue

[{'id': '375000667413_0',
  'arrival_time': 603026300,
  'service_time': 418.51043701171795},
 {'id': '375000669289_0',
  'arrival_time': 606413041,
  'service_time': 249.68147277831997},
 {'id': '375000670586_0',
  'arrival_time': 608994453,
  'service_time': 241.75643920898398},
 {'id': '375000670588_0',
  'arrival_time': 608994466,
  'service_time': 195.52230834960898}]

In [289]:
my_list = [x["arrival_time"] for x in x.queue]
time_deltas = [my_list[i]-my_list[i-1] if i !=0 else 0 for i in reversed(range(len(my_list)))][::-1]
time_deltas

[0, 3386741, 2581412, 13]

In [223]:
times = x.exit_time()
times 

{'375000667413_0': [603026300, 418.51043701171795, 603026300],
 '375000669289_0': [606413041, 249.68147277831997, 1209439341],
 '375000670586_0': [608994453, 241.75643920898398, 1818433794],
 '375000670588_0': [608994466, 195.52230834960898, 2427428260]}

In [224]:
times == x.queue_status

True

In [225]:
x.queue

[{'id': '375000667413_0',
  'arrival_time': 603026300,
  'service_time': 418.51043701171795},
 {'id': '375000669289_0',
  'arrival_time': 606413041,
  'service_time': 249.68147277831997},
 {'id': '375000670586_0',
  'arrival_time': 608994453,
  'service_time': 241.75643920898398},
 {'id': '375000670588_0',
  'arrival_time': 608994466,
  'service_time': 195.52230834960898}]

In [227]:
x.turn_around_times()

{'375000667413_0': (603026300, 418.51043701171795, 603026300, 0),
 '375000669289_0': (606413041, 249.68147277831997, 1209439341, 603026300),
 '375000670586_0': (608994453, 241.75643920898398, 1818433794, 1209439341),
 '375000670588_0': (608994466, 195.52230834960898, 2427428260, 1818433794)}

In [ ]:
x.turn_around_times()

[0, 603026300, 1209439341, 1818433794]

In [228]:
x.waiting_times()

{'375000667413_0': (603026300,
  418.51043701171795,
  603026300,
  0,
  -418.51043701171795),
 '375000669289_0': (606413041,
  249.68147277831997,
  1209439341,
  603026300,
  603026050.3185272),
 '375000670586_0': (608994453,
  241.75643920898398,
  1818433794,
  1209439341,
  1209439099.2435608),
 '375000670588_0': (608994466,
  195.52230834960898,
  2427428260,
  1818433794,
  1818433598.4776917)}

In [ ]:
603026300 + 606413041

1209439341

In [ ]:

times = []

for n, task in enumerate(x.queue):
    
    if n == 0:
        times.append(task['arrival_time'])

    else:
        times.append(times[n-1] + task['arrival_time'])

times

[603026300, 1209439341, 1818433794]

In [ ]:
#from NBD.module import FCFS

class FCFS:

################################################################
    
    def exit_time(dizionario : dict): 
        # cumulative execution times

        times = []
        
        for n,i in enumerate(dizionario.keys()):
            
            if n == 0:
                times.append(dizionario[i][1])
        
            else:
                times.append(times[n-1] + dizionario[i][1])
        
        new_diz = {v[0]:(*v[1], x) for v, x in zip(dizionario.items(), times)}
        
        return new_diz
#########################################################################################

    def turn_around_times(dizionario : dict):   #
        
        a_list = []
        
        for i in dizionario.keys():

            desired = dizionario[i][2] - dizionario[i][0]
            # difference between Exit time and Arrival of the processes

            #if desired < 0: 
            #    desired = dizionario[i][1]
            # if negative there is no queue ahead, so TAT = Burst_time
            # perche WT = TAT - Burst_time deve essere  0
            # not sure 

            a_list.append(desired)

        new_diz = {v[0]:(*v[1], x) for v, x in zip(dizionario.items(), a_list)}

        return new_diz
#########################################################################################

    def waiting_times(dizionario : dict):
        
        a_list = []
        
        for i in dizionario.keys():
            
            wt = dizionario[i][3]-dizionario[i][1]
            # difference between turn around time and burst time

            a_list.append(wt)
        
        new_diz = {v[0]:(*v[1], x) for v, x in zip(dizionario.items(), a_list)}
        
        return new_diz
#########################################################################################
    
    def Scheduling(queue : list, µ = 0.1):   # queue is list of dict

        arrival_time_tasks = [int(queue[i]["Arrival_Time"]) for i in range(len(queue))]
        running_time_tasks = [(queue[i]["CPU"]/ µ)*1000 for i in range(len(queue))]      # from seconds to milliseconds

        dict_values = list(zip(arrival_time_tasks, running_time_tasks))

        #d = {int(queue[key]["Job_ID"]): dict_values[key] for key in range(len(queue))}
        d = {str(int(queue[key]["Job_ID"]))+ "_" + str(int(queue[key]["Task_ID"])): dict_values[key] for key in range(len(queue))}
        d = dict(sorted(d.items(), key=lambda item: item[1])) # sort by time arrival (not sure if needed)
        # KEY = job_id --> VALUES = Arrival_Time, Burst_time

        my_list = [x[0] for x in d.values()]
        time_deltas = [my_list[i]-my_list[i-1] if i !=0 else 0 for i in reversed(range(len(my_list)))][::-1]
        new_d = {v[0]:(new,v[1][1]) for v, new in zip(d.items(), time_deltas)}
        # KEY = job_id --> VALUES = Time_delta, Burst_time

        results = FCFS.exit_time(new_d)
        results = FCFS.turn_around_times(results)
        results = FCFS.waiting_times(results)
        # KEY = job_id --> VALUES = Time_delta, Burst_time, Exit_time, Turn_around_time, Waiting_time

        df = pd.DataFrame.from_dict(results, orient='index').reset_index()
        df.columns = ["Job_id", "Time_deltas", "execution_time", "exit_time", "turn_around", "waiting"]
        
        #print("\n Mean Waiting Time: {} \n".format(df["waiting"].mean()))

        return df
        
#########################################################################################

In [ ]:
# fake queue for testing purposes

server_dict = [data.iloc[i].to_dict() for i in range(20)]

In [ ]:
def highlight_max(cell):

    if type(cell) != str and cell < 0 :

        return 'color: red'
    else:
        return 'color: green'

In [ ]:
FCFS.Scheduling(server_dict).style.applymap(highlight_max, subset=pd.IndexSlice[:, ["execution_time", 'waiting']])

# negative waiting_time --> no queue for this job          --> that's good i guess
#                     --> server stays jobless for a while --> maybe bad 

,Job_id,Time_deltas,execution_time,exit_time,turn_around,waiting
0,375000667413_0,0,418.510437,418.510437,418.510437,0.000000
1,375000669289_0,3386741,249.681473,668.191910,-3386072.808090,-3386322.489563
2,375000670586_0,2581412,241.756439,909.948349,-2580502.051651,-2580743.808090
3,375000670588_0,13,195.522308,1105.470657,1092.470657,896.948349
4,375000670590_0,48437,280.437469,1385.908127,-47051.091873,-47331.529343
5,375000670901_0,751754,279.102325,1665.010452,-750088.989548,-750368.091873
6,375000670926_0,28007,303.726196,1968.736649,-26038.263351,-26341.989548
7,375000670981_0,412680,184.736252,2153.472900,-410526.527100,-410711.263351
8,375000671303_0,747053,250.272751,2403.745651,-744649.254349,-744899.527100
9,375000671907_0,4614372,87446.479797,89850.225449,-4524521.774551,-4611968.254349


_______________________________________________________

In [ ]:
######################################################### 

def random_dispatcher(dizionario):  #diz da assegnare alle coda

    server_selected = rdm.randrange(0, N)
    
    if servers_queues[server_selected] == None:
      servers_queues[server_selected] = [dizionario]
    else:
      servers_queues[server_selected].append(dizionario)
    

In [ ]:
servers_queues =  {x:None for x in range(0,N)}
#print(servers_queues[0])

st = time.time()

for i in tqdm(range(len(data))):

    diz = data.iloc[i].to_dict()

    random_dispatcher(diz)

et = time.time()
print('\n Execution time:', str(datetime.timedelta(seconds = et - st))[:10] , 'mins')

  0%|          | 0/2329133 [00:00<?, ?it/s]


 Execution time: 0:03:52.60 mins


In [ ]:
print("each q length:", [len(x) for x in servers_queues.values()])

sum([len(x) for x in servers_queues.values()]) == data.shape[0]

each q length: [36292, 36786, 36377, 36365, 36287, 36549, 36303, 36480, 36627, 36218, 36114, 36292, 36097, 36455, 36559, 36448, 36652, 36643, 35942, 36247, 36168, 36255, 36351, 36629, 36390, 36560, 36461, 36292, 36490, 36532, 36369, 36482, 36275, 36264, 36262, 35968, 36141, 36530, 36474, 36189, 36560, 36416, 36408, 36731, 36222, 36396, 36441, 36449, 36620, 36202, 36580, 36510, 36405, 36291, 36354, 36169, 36572, 36203, 36032, 36264, 36700, 36461, 36510, 36822]


True

In [ ]:
list_of_df = []

for key in tqdm(servers_queues.keys()):

  x = FCFS.Scheduling(servers_queues[key])
  
  list_of_df.append(x)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
## TO DO:
# re-group by Job_ID and evaluate Job response time
# then Job slowdown, Utilization coefficient of server,  Messaging load (?) 

In [ ]:
df = pd.concat(list_of_df)
df = df.sort_values(by="Job_id")

print("\n df.shape: {} \n".format(df.shape))   # 1163061

display(df.head())

len(set(df["Job_id"])) == 1163061  # stats.shape[0]


 df.shape: (2329133, 6) 



,Job_id,Time_deltas,execution_time,exit_time,turn_around,waiting
0,375000667413_0,0,418.510437,418.510437,418.510437,0.0
0,375000669289_0,0,249.681473,249.681473,249.681473,0.0
0,375000670586_0,0,241.756439,241.756439,241.756439,0.0
0,375000670588_0,0,195.522308,195.522308,195.522308,0.0
0,375000670590_0,0,280.437469,280.437469,280.437469,0.0


False

In [ ]:
df.shape[0] == data.shape[0]

True

In [ ]:
# JOB_RESPONSE_TIME = execution_time + waiting_time for all its task 

# ma se due task dello stesso job sono svolte in contemporanea su due server diversi?
# c'è da riprendere il tempo assoluto di arrivo 

In [ ]:
df.groupby(['Job_id'])[["waiting", "exit_time"]].sum()

,waiting,exit_time
Job_id,,
375000667413_0,0.000000e+00,4.185104e+02
375000669289_0,0.000000e+00,2.496815e+02
375000670586_0,0.000000e+00,2.417564e+02
375000670588_0,0.000000e+00,1.955223e+02
375000670590_0,0.000000e+00,2.804375e+02
...,...,...
400465219425_0,9.377879e+08,9.726386e+08
400465219425_1,1.340268e+09,1.367037e+09
400465256347_0,7.752852e+08,1.529105e+09
